# Code Mainly from From Tonmoy Discussion code

In [2]:
#Import packages
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Dropout
from keras.layers import Conv2D,LSTM,BatchNormalization,MaxPooling2D,Reshape
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import sys
%cd /content/drive/MyDrive/ECE C247 Final

Mounted at /content/drive
/content/drive/MyDrive/ECE C247 Final


In [4]:
# Read in the dataset
%cd /content/drive/MyDrive/ECE C247 Final/Dataset
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

/content/drive/MyDrive/ECE C247 Final/Dataset


In [5]:
def data_prep(X,y,sub_sample,average,noise):
    
    total_X = None
    total_y = None
    
    # Trimming the data (sample,22,1000) -> (sample,22,500)
    X = X[:,:,0:500]
    print('Shape of X after trimming:',X.shape)
    
    # Maxpooling the data (sample,22,1000) -> (sample,22,500/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)
    
    
    total_X = X_max
    total_y = y
    print('Shape of X after maxpooling:',total_X.shape)
    
    # Averaging + noise 
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)
    
    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    print('Shape of X after averaging+noise and concatenating:',total_X.shape)
    
    # Subsampling
    
    for i in range(sub_sample):
        
        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)
            
        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))
        
    
    print('Shape of X after subsampling and concatenating:',total_X.shape)
    return total_X,total_y

In [6]:
## Adjusting the labels so that 

# Cue onset left - 0
# Cue onset right - 1
# Cue onset foot - 2
# Cue onset tongue - 3

y_train_valid -= 769
y_test -= 769


## Random splitting and reshaping the data
# First generating the training and validation indices using random splitting

ind_valid = np.random.choice(2115, 375, replace=False)
ind_train = np.array(list(set(range(2115)).difference(set(ind_valid))))

# Creating the training and validation sets using the generated indices
(X_train, X_valid) = X_train_valid[ind_train], X_train_valid[ind_valid] 
(y_train, y_valid) = y_train_valid[ind_train], y_train_valid[ind_valid]


## Preprocessing the dataset
x_train,y_train = data_prep(X_train,y_train,2,2,True)
x_valid,y_valid = data_prep(X_valid,y_valid,2,2,True)
X_test_prep,y_test_prep = data_prep(X_test,y_test,2,2,True)


print('Shape of training set:',x_train.shape)
print('Shape of validation set:',x_valid.shape)
print('Shape of training labels:',y_train.shape)
print('Shape of validation labels:',y_valid.shape)
print('Shape of testing set:',X_test_prep.shape)
print('Shape of testing labels:',y_test_prep.shape)


# Converting the labels to categorical variables for multiclass classification
y_train = to_categorical(y_train, 4)
y_valid = to_categorical(y_valid, 4)
y_test = to_categorical(y_test_prep, 4)
print('Shape of training labels after categorical conversion:',y_train.shape)
print('Shape of validation labels after categorical conversion:',y_valid.shape)
print('Shape of test labels after categorical conversion:',y_test.shape)

# Adding width of the segment to be 1
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)
print('Shape of training set after adding width info:',x_train.shape)
print('Shape of validation set after adding width info:',x_valid.shape)
print('Shape of test set after adding width info:',x_test.shape)


# Reshaping the training and validation dataset
x_train = np.swapaxes(x_train, 1,3)
x_train = np.swapaxes(x_train, 1,2)
x_valid = np.swapaxes(x_valid, 1,3)
x_valid = np.swapaxes(x_valid, 1,2)
x_test = np.swapaxes(x_test, 1,3)
x_test = np.swapaxes(x_test, 1,2)
print('Shape of training set after dimension reshaping:',x_train.shape)
print('Shape of validation set after dimension reshaping:',x_valid.shape)
print('Shape of test set after dimension reshaping:',x_test.shape)

Shape of X after trimming: (1740, 22, 500)
Shape of X after maxpooling: (1740, 22, 250)
Shape of X after averaging+noise and concatenating: (3480, 22, 250)
Shape of X after subsampling and concatenating: (6960, 22, 250)
Shape of X after trimming: (375, 22, 500)
Shape of X after maxpooling: (375, 22, 250)
Shape of X after averaging+noise and concatenating: (750, 22, 250)
Shape of X after subsampling and concatenating: (1500, 22, 250)
Shape of X after trimming: (443, 22, 500)
Shape of X after maxpooling: (443, 22, 250)
Shape of X after averaging+noise and concatenating: (886, 22, 250)
Shape of X after subsampling and concatenating: (1772, 22, 250)
Shape of training set: (6960, 22, 250)
Shape of validation set: (1500, 22, 250)
Shape of training labels: (6960,)
Shape of validation labels: (1500,)
Shape of testing set: (1772, 22, 250)
Shape of testing labels: (1772,)
Shape of training labels after categorical conversion: (6960, 4)
Shape of validation labels after categorical conversion: (15

In [6]:
y_train.shape

(6960, 4)

In [7]:
# Building the CNN model using sequential class
hybrid_cnn_lstm_model = Sequential()

# Conv. block 1
hybrid_cnn_lstm_model.add(Conv2D(filters=25, kernel_size=(10,1), padding='same', activation='elu', input_shape=(250,1,22)))
hybrid_cnn_lstm_model.add(MaxPooling2D(pool_size=(3,1), padding='same')) # Read the keras documentation
hybrid_cnn_lstm_model.add(BatchNormalization())
hybrid_cnn_lstm_model.add(Dropout(0.5))

# Conv. block 2
hybrid_cnn_lstm_model.add(Conv2D(filters=50, kernel_size=(10,1), padding='same', activation='elu'))
hybrid_cnn_lstm_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
hybrid_cnn_lstm_model.add(BatchNormalization())
hybrid_cnn_lstm_model.add(Dropout(0.5))

# Conv. block 3
hybrid_cnn_lstm_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu'))
hybrid_cnn_lstm_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
hybrid_cnn_lstm_model.add(BatchNormalization())
hybrid_cnn_lstm_model.add(Dropout(0.5))

# Conv. block 4
hybrid_cnn_lstm_model.add(Conv2D(filters=200, kernel_size=(10,1), padding='same', activation='elu'))
hybrid_cnn_lstm_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
hybrid_cnn_lstm_model.add(BatchNormalization())
hybrid_cnn_lstm_model.add(Dropout(0.5))

# FC+LSTM layers
hybrid_cnn_lstm_model.add(Flatten()) # Adding a flattening operation to the output of CNN block
hybrid_cnn_lstm_model.add(Dense((100))) # FC layer with 100 units
hybrid_cnn_lstm_model.add(Reshape((100,1))) # Reshape my output of FC layer so that it's compatible
hybrid_cnn_lstm_model.add(LSTM(10, dropout=0.6, recurrent_dropout=0.1, input_shape=(100,1), return_sequences=False))


# Output layer with Softmax activation 
hybrid_cnn_lstm_model.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation


# Printing the model summary
hybrid_cnn_lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 250, 1, 25)        5525      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 84, 1, 25)        0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 84, 1, 25)        100       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 84, 1, 25)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 84, 1, 50)         12550     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 1, 50)        0

This is a sequential model that consists of convolutional layers, max-pooling layers, batch normalization layers, dropout layers, a dense layer, an LSTM layer, and a final dense layer. The model takes 250x1 input shape and has the output shape of 4 classes. The convolutional layers are used to extract features from the input data. The max-pooling layers are used to downsample the output of the convolutional layers. The batch normalization layers are used to normalize the inputs of each layer, which helps to stabilize the training process. The dropout layers are used to randomly drop out some units during training, which helps to prevent overfitting. The dense layer is used to perform classification after the convolutional layers. The LSTM layer is used to capture the temporal relationships in the input data. Finally, the last dense layer is used to produce the output of the model, which is a probability distribution over the four classes. The model has 350,499 parameters, out of which 349,749 are trainable and the remaining 750 are non-trainable.

## (CNN-LSTM) Defining the hyperparameters of the hybrid CNN-LSTM model

In [8]:
# Model parameters
learning_rate = 1e-3
epochs = 50
hybrid_cnn_lstm_optimizer = keras.optimizers.Adam(lr=learning_rate)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## Apply the grid-search to find the optimized hyperparameters:


In [11]:
x_train.shape

(6960, 250, 1, 22)

In [9]:
# from sklearn.model_selection import GridSearchCV
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.optimizers import Adam

# # Define the hyperparameters to search over
# learning_rates = [0.001, 0.0001]
# dropout_rates = [0.2, 0.5]

# # Define a function to build the model with the specified hyperparameters
# def build_model(learning_rate=0.001, dropout_rate=0.5, filter_size=32, kernel_size=(10,1), lstm_units=10):
#     model = Sequential()
#     model.add(Conv2D(filters=filter_size, kernel_size=kernel_size, padding='same', activation='elu', input_shape=(250,1,22)))
#     model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Dropout(dropout_rate))

#     model.add(Conv2D(filters=filter_size*2, kernel_size=kernel_size, padding='same', activation='elu'))
#     model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Dropout(dropout_rate))

#     model.add(Conv2D(filters=filter_size*4, kernel_size=kernel_size, padding='same', activation='elu'))
#     model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Dropout(dropout_rate))

#     model.add(Conv2D(filters=filter_size*8, kernel_size=kernel_size, padding='same', activation='elu'))
#     model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
#     model.add(BatchNormalization())
#     model.add(Dropout(dropout_rate))

#     model.add(Flatten())
#     model.add(Dense(100))
#     model.add(Reshape((100,1)))
#     model.add(LSTM(lstm_units, dropout=dropout_rate, recurrent_dropout=dropout_rate))
#     model.add(Dense(4, activation='softmax'))

#     optimizer = Adam(lr=learning_rate)
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#     return model

# # Create the GridSearchCV object
# param_grid = {'learning_rate': learning_rates,
#               'dropout_rate': dropout_rates}
# grid_search = GridSearchCV(estimator=KerasClassifier(build_fn=build_model, epochs=50, batch_size=32), param_grid=param_grid, cv=3)

# # Fit the GridSearchCV object to the data
# grid_search.fit(x_train, y_train)

# # Print the best hyperparameters and corresponding model performance
# print("Best hyperparameters: ", grid_search.best_params_)
# print("Best cross-validation score: ", grid_search.best_score_)


<ipython-input-9-995e67384231>:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_search = GridSearchCV(estimator=KerasClassifier(build_fn=build_model, epochs=50, batch_size=32), param_grid=param_grid, cv=3)
/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
145/145 [==============================] - 32s 200ms/step - loss: 1.3281 - accuracy: 0.3603
Epoch 2/50
145/145 [==============================] - 30s 204ms/step - loss: 1.1989 - accuracy: 0.4812
Epoch 3/50
145/145 [==============================] - 30s 203ms/step - loss: 1.0809 - accuracy: 0.5448
Epoch 4/50
145/145 [==============================] - 29s 200ms/step - loss: 0.9708 - accuracy: 0.5909
Epoch 5/50
145/145 [==============================] - 29s 198ms/step - loss: 0.8654 - accuracy: 0.6498
Epoch 6/50
145/145 [==============================] - 29s 200ms/step - loss: 0.7626 - accuracy: 0.6909
Epoch 7/50
145/145 [==============================] - 29s 197ms/step - loss: 0.6655 - accuracy: 0.7412
Epoch 8/50
145/145 [==============================] - 29s 201ms/step - loss: 0.5693 - accuracy: 0.7978
Epoch 9/50
145/145 [==============================] - 29s 200ms/step - loss: 0.5033 - accuracy: 0.8297
Epoch 10/50
145/145 [==============================] - 30s 204ms/step - l

Epoch 1/50
145/145 [==============================] - 31s 193ms/step - loss: 1.3332 - accuracy: 0.3345
Epoch 2/50
145/145 [==============================] - 28s 194ms/step - loss: 1.1834 - accuracy: 0.4845
Epoch 3/50
145/145 [==============================] - 28s 194ms/step - loss: 1.0401 - accuracy: 0.5614
Epoch 4/50
145/145 [==============================] - 28s 193ms/step - loss: 0.9247 - accuracy: 0.6267
Epoch 5/50
145/145 [==============================] - 28s 194ms/step - loss: 0.7705 - accuracy: 0.7017
Epoch 6/50
145/145 [==============================] - 28s 193ms/step - loss: 0.6685 - accuracy: 0.7489
Epoch 7/50
145/145 [==============================] - 28s 194ms/step - loss: 0.5581 - accuracy: 0.8097
Epoch 8/50
145/145 [==============================] - 28s 195ms/step - loss: 0.4494 - accuracy: 0.8431
Epoch 9/50
145/145 [==============================] - 28s 193ms/step - loss: 0.3955 - accuracy: 0.8675
Epoch 10/50
145/145 [==============================] - 29s 197ms/step - l

Epoch 1/50
145/145 [==============================] - 30s 195ms/step - loss: 1.3693 - accuracy: 0.2961
Epoch 2/50
145/145 [==============================] - 28s 196ms/step - loss: 1.2781 - accuracy: 0.4034
Epoch 3/50
145/145 [==============================] - 28s 194ms/step - loss: 1.1913 - accuracy: 0.4414
Epoch 4/50
145/145 [==============================] - 28s 195ms/step - loss: 1.1085 - accuracy: 0.5179
Epoch 5/50
145/145 [==============================] - 28s 196ms/step - loss: 0.9685 - accuracy: 0.5946
Epoch 6/50
145/145 [==============================] - 29s 200ms/step - loss: 0.8229 - accuracy: 0.6647
Epoch 7/50
145/145 [==============================] - 29s 196ms/step - loss: 0.6922 - accuracy: 0.7252
Epoch 8/50
145/145 [==============================] - 29s 197ms/step - loss: 0.5972 - accuracy: 0.7845
Epoch 9/50
145/145 [==============================] - 28s 195ms/step - loss: 0.5203 - accuracy: 0.8099
Epoch 10/50
145/145 [==============================] - 28s 195ms/step - l

Epoch 1/50
145/145 [==============================] - 31s 198ms/step - loss: 1.3683 - accuracy: 0.2892
Epoch 2/50
145/145 [==============================] - 29s 200ms/step - loss: 1.3142 - accuracy: 0.3741
Epoch 3/50
145/145 [==============================] - 29s 199ms/step - loss: 1.2675 - accuracy: 0.4017
Epoch 4/50
145/145 [==============================] - 29s 203ms/step - loss: 1.2210 - accuracy: 0.4470
Epoch 5/50
145/145 [==============================] - 29s 201ms/step - loss: 1.1796 - accuracy: 0.5037
Epoch 6/50
145/145 [==============================] - 29s 199ms/step - loss: 1.1319 - accuracy: 0.5481
Epoch 7/50
145/145 [==============================] - 29s 197ms/step - loss: 1.0965 - accuracy: 0.5722
Epoch 8/50
145/145 [==============================] - 29s 198ms/step - loss: 1.0605 - accuracy: 0.5899
Epoch 9/50
145/145 [==============================] - 28s 196ms/step - loss: 1.0059 - accuracy: 0.6177
Epoch 10/50
145/145 [==============================] - 29s 197ms/step - l

...

Epoch 1/50
145/145 [==============================] - 32s 203ms/step - loss: 1.3909 - accuracy: 0.2528
Epoch 2/50
145/145 [==============================] - 30s 203ms/step - loss: 1.3926 - accuracy: 0.2524
Epoch 3/50
145/145 [==============================] - 29s 200ms/step - loss: 1.3833 - accuracy: 0.2677
Epoch 4/50
145/145 [==============================] - 29s 199ms/step - loss: 1.3763 - accuracy: 0.2877
Epoch 5/50
145/145 [==============================] - 29s 199ms/step - loss: 1.3769 - accuracy: 0.2817
Epoch 6/50
145/145 [==============================] - 29s 199ms/step - loss: 1.3648 - accuracy: 0.3011
Epoch 7/50
145/145 [==============================] - 29s 199ms/step - loss: 1.3569 - accuracy: 0.3071
Epoch 8/50
145/145 [==============================] - 30s 206ms/step - loss: 1.3440 - accuracy: 0.3366
Epoch 9/50
145/145 [==============================] - 30s 204ms/step - loss: 1.3323 - accuracy: 0.3621
Epoch 10/50
145/145 [==============================] - 29s 202ms/step - l

Epoch 1/50
218/218 [==============================] - 102s 456ms/step - loss: 1.2703 - accuracy: 0.4148
Epoch 2/50
218/218 [==============================] - 101s 465ms/step - loss: 1.1251 - accuracy: 0.5112
Epoch 3/50
218/218 [==============================] - 100s 458ms/step - loss: 0.9709 - accuracy: 0.5904
Epoch 4/50
218/218 [==============================] - 101s 464ms/step - loss: 0.8022 - accuracy: 0.6773
Epoch 5/50
218/218 [==============================] - 100s 457ms/step - loss: 0.6623 - accuracy: 0.7480
Epoch 6/50
218/218 [==============================] - 99s 456ms/step - loss: 0.5059 - accuracy: 0.8167
Epoch 7/50
218/218 [==============================] - 101s 461ms/step - loss: 0.4026 - accuracy: 0.8690
Epoch 8/50
218/218 [==============================] - 99s 456ms/step - loss: 0.3174 - accuracy: 0.8994
Epoch 9/50
218/218 [==============================] - 101s 466ms/step - loss: 0.2798 - accuracy: 0.9128
Epoch 10/50
218/218 [==============================] - 99s 455ms/s

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-995e67384231>", line 49, in <module>
    grid_search.fit(x_train, y_train)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py", line 909, in fit
    self.best_estimator_.fit(X, y, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/keras/wrappers/scikit_learn.py", line 248, in fit
    return super().fit(x, y, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras/wrappers/scikit_learn.py", line 175, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/usr/local/lib/python3.9/dist-packages/keras/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras/engine/training.py", line 1650, in fit
    tmp_logs = self.train_function(iterator)
  

KeyboardInterrupt: ignored

In [ ]:
# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_
# best_score = grid_search.best_score_

## (CNN-LSTM) Compiling, training and validating the model

In [ ]:
# Compiling the model
hybrid_cnn_lstm_model.compile(loss='categorical_crossentropy',
                 optimizer=hybrid_cnn_lstm_optimizer,
                 metrics=['accuracy'])

# Training and validating the model
hybrid_cnn_lstm_model_results = hybrid_cnn_lstm_model.fit(x_train,
             y_train,
             batch_size=64,
             epochs=epochs,
             validation_data=(x_valid, y_valid), verbose=True)


## (CNN-LSTM) Visualizing the accuracy and loss trajectory

In [ ]:
import matplotlib.pyplot as plt

# Plotting accuracy trajectory
plt.plot(hybrid_cnn_lstm_model_results.history['accuracy'])
plt.plot(hybrid_cnn_lstm_model_results.history['val_accuracy'])
plt.title('Hybrid CNN-LSTM model accuracy trajectory')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Plotting loss trajectory
plt.plot(hybrid_cnn_lstm_model_results.history['loss'],'o')
plt.plot(hybrid_cnn_lstm_model_results.history['val_loss'],'o')
plt.title('Hybrid CNN-LSTM model loss trajectory')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## (CNN-LSTM) Testing the performance of the hybrid CNN-LSTM model on the held out test set

In [ ]:
## Testing the hybrid CNN-LSTM model

hybrid_cnn_lstm_score = hybrid_cnn_lstm_model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy of the hybrid CNN-LSTM model:',hybrid_cnn_lstm_score[1])

1. Better feature extraction: The convolutional layers of the CNN are able to learn spatial features from the input data, while the LSTM layer can capture temporal dependencies in the sequence. This combination of both types of layers can help to extract meaningful features from the data and improve the accuracy of the model.
2. Robust to input sequence length: LSTMs are designed to handle variable-length sequences, which makes them a good choice for working with EEG data that may have varying lengths. This allows the model to learn patterns in EEG signals that may vary in duration, such as those related to certain types of brain activity.
3. Handles noise and variability: EEG data can be very noisy and contain a lot of variability due to differences in electrode placement and individual differences between subjects. The CNN+LSTM model can help to handle this noise and variability by learning robust features from the data and capturing the temporal dependencies in the sequence.
4. Good for real-time applications: The CNN+LSTM model can be used for real-time EEG signal processing, which makes it a good choice for $\mathrm{BCl}$ applications where timely and accurate predictions are important.


Overall, the CNN+LSTM model has shown promise in EEG classification tasks and can potentially lead to more accurate and robust models for $\mathrm{BCl}$ applications.

In [ ]:
train_scores = []
test_scores = []
time_period = 1000
for sub_idx in range(9):
    print("------------------------------------")
    print("Subject: %d" %(sub_idx))
    X_train_cur = x_train[:, :time_period, :]
    y_train_cur = y_train
    X_test_cur = x_test[:, :time_period, :]
    y_test_cur = y_test

    # single subject dataset
    X_train_cur = X_train_cur[np.where(person_train_valid == sub_idx)[0]]
    y_train_cur = y_train_cur[np.where(person_train_valid == sub_idx)[0]]
    X_test_cur = X_test_cur[np.where(person_test == sub_idx)[0]]
    y_test_cur = y_test_cur[np.where(person_test == sub_idx)[0]]

    #lecun = lecun_uniform(seed=42)

    #model = model_type(lecun, time_period = 1000)
    history1 = hybrid_cnn_lstm_model.fit(X_train_cur,y_train_cur,batch_size=38,epochs=38,shuffle=True,validation_data=(X_test_cur, y_test_cur))

    #history = model.fit(X_train_cur, y_train_cur, epochs=38, batch_size=32, validation_data=(X_test_cur, y_test_cur), shuffle=True, verbose=1)
    train_score = hybrid_cnn_lstm_model.evaluate(X_train_cur, y_train_cur)
    test_score = hybrid_cnn_lstm_model.evaluate(X_test_cur, y_test_cur)
    train_scores.append(train_score[1]*100)
    test_scores.append(test_score[1]*100)


In [ ]:
plt.figure()
plt.bar(range(1,10), train_scores, color = 'deepskyblue', label='train')
plt.xlabel('Subject')
plt.legend()
plt.title("Train Accuracies of CNN + LSTM Versus Subject")
plt.show()

plt.figure()
plt.bar(range(1,10), test_scores, color = 'darkorange' ,label='test')
plt.xlabel('Subject')
plt.legend()
plt.title("Test Accuracies of CNN + LSTM Versus Subject")
plt.show()

print('Train Accuracy:')
print(train_scores)
print('Test Accuracy')
print(test_scores)